In [ ]:
!pip install https://github.com/GoogleCloudPlatform/healthcare-api-dicomweb-cli/archive/v1.0.zip

from pathlib import Path
import os

from google.auth.transport import requests
from google.oauth2 import service_account

def dicomweb_search_instance(project_id, location, dataset_id, dicom_store_id):
    """Handles the GET requests specified in DICOMweb standard.

    See https://github.com/GoogleCloudPlatform/python-docs-samples/tree/main/healthcare/api-client/v1/dicom
    before running the sample."""


    os.environ["GOOGLE_APPLICATION_CREDENTIALS"]="/kaapana/mounted/workflows/dags/credentials.json"

    if not Path(os.environ["GOOGLE_APPLICATION_CREDENTIALS"]).exists():
        print("ERROR, CREDENTIALS FILE IS NOT LOCATED")
        print(os.getcwd())
        print(os.environ["GOOGLE_APPLICATION_CREDENTIALS"])

    credentials = service_account.Credentials.from_service_account_file(
        os.environ["GOOGLE_APPLICATION_CREDENTIALS"]
    )

    scoped_credentials = credentials.with_scopes(
        ["https://www.googleapis.com/auth/cloud-platform"]
    )
    session = requests.AuthorizedSession(scoped_credentials)
    base_url = "https://healthcare.googleapis.com/v1"
    url = f"{base_url}/projects/{project_id}/locations/{location}"
    dicomweb_path = "{}/datasets/{}/dicomStores/{}/dicomWeb/instances".format(
        url, dataset_id, dicom_store_id
    )
    headers = {"Content-Type": "application/dicom+json; charset=utf-8"}

    response = session.get(dicomweb_path, headers=headers)
    response.raise_for_status()
    instances = response.json()

    return instances

project_id = "idc-external-031"
location = "europe-west2"
dataset_id = "kaapana-integration-test"
dicom_store_id = "kaapana-integration-test-store"

all_metadata_dict = dicomweb_search_instance(
    project_id=project_id,
    location=location,
    dataset_id= dataset_id,
    dicom_store_id = dicom_store_id
)

all_metadata_dict[0]